# 🏪 서울시 편의점 매출 분석 - EDA
- **기간**: 2022년 1분기 ~ 2025년 3분기
- **목적**: 데이터 양이 회귀분석에 적합한지 확인

## 1. 데이터 로드

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정 (안티그래비티용)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료!")

In [ ]:
# 연도별 데이터 로드 (인코딩: cp949)
df_2022 = pd.read_csv('data/sales_2022.csv', encoding='cp949')
df_2023 = pd.read_csv('data/sales_2023.csv', encoding='cp949')
df_2024 = pd.read_csv('data/sales_2024.csv', encoding='cp949')
df_2025 = pd.read_csv('data/sales_2025.csv', encoding='cp949')

# 전체 합치기
df = pd.concat([df_2022, df_2023, df_2024, df_2025], ignore_index=True)
print(f"✅ 데이터 로드 완료! 총 {len(df):,}개 row")

## 2. 전체 데이터 개요

In [ ]:
print("📊 전체 데이터 개요")
print("=" * 50)
print(f"전체 row 수: {len(df):,}개")
print(f"컬럼 수: {len(df.columns)}개")
print(f"총 분기 수: {df['기준_년분기_코드'].nunique()}개")
print(f"행정동 수: {df['행정동_코드'].nunique()}개")
print(f"업종 수: {df['서비스_업종_코드_명'].nunique()}개")

In [ ]:
# 기간 확인
print("📅 분기 목록:")
print(sorted(df['기준_년분기_코드'].unique()))

## 3. 편의점 데이터 필터링

In [ ]:
# 편의점만 필터링
df_cvs = df[df['서비스_업종_코드_명'] == '편의점'].copy()

print("🏪 편의점 데이터")
print("=" * 50)
print(f"편의점 row 수: {len(df_cvs):,}개")
print(f"행정동 수: {df_cvs['행정동_코드'].nunique()}개")
print(f"분기 수: {df_cvs['기준_년분기_코드'].nunique()}개")

## 4. 분기별 데이터 개수

In [ ]:
print("📅 분기별 데이터 개수")
print("=" * 50)
quarter_counts = df_cvs['기준_년분기_코드'].value_counts().sort_index()
print(quarter_counts)

## 5. 기초 통계

In [ ]:
print("📈 기초 통계 (편의점 당월 매출)")
print("=" * 50)
df_cvs['당월_매출_금액'].describe()

## 6. 결측치 확인

In [ ]:
print("🔍 결측치 확인")
print("=" * 50)
null_counts = df_cvs.isnull().sum()
null_pct = (null_counts / len(df_cvs) * 100).round(2)
null_df = pd.DataFrame({'결측치 수': null_counts, '비율(%)': null_pct})
null_with_values = null_df[null_df['결측치 수'] > 0]

if len(null_with_values) == 0:
    print("✅ 결측치 없음!")
else:
    print(null_with_values)

## 7. 데이터 양 적합성 판단

In [ ]:
print("🎯 데이터 양 적합성 판단")
print("=" * 50)

row_count = len(df_cvs)
quarter_count = df_cvs['기준_년분기_코드'].nunique()
dong_count = df_cvs['행정동_코드'].nunique()
null_pct_sales = df_cvs['당월_매출_금액'].isnull().sum() / len(df_cvs) * 100

# 판단
print(f"{'✅' if row_count >= 1000 else '❌'} 데이터 양: {row_count:,}개 (기준: 1,000개 이상)")
print(f"{'✅' if quarter_count >= 8 else '❌'} 기간: {quarter_count}개 분기 (기준: 8개 이상)")
print(f"{'✅' if null_pct_sales < 10 else '❌'} 결측치: {null_pct_sales:.1f}% (기준: 10% 미만)")

## 8. 시각화 - 분기별 매출 추이

In [ ]:
# 분기별 평균 매출
quarterly_sales = df_cvs.groupby('기준_년분기_코드')['당월_매출_금액'].mean()

plt.figure(figsize=(12, 5))
plt.plot(quarterly_sales.index.astype(str), quarterly_sales.values / 1e8, marker='o', linewidth=2, markersize=8)
plt.title('편의점 분기별 평균 매출 추이', fontsize=14)
plt.xlabel('분기', fontsize=12)
plt.ylabel('평균 매출 (억원)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. 매출 분포 확인

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(df_cvs['당월_매출_금액'] / 1e8, bins=50, edgecolor='black', alpha=0.7)
plt.title('편의점 매출 분포', fontsize=14)
plt.xlabel('매출 (억원)', fontsize=12)
plt.ylabel('빈도', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 📝 최종 결론

In [ ]:
print("=" * 60)
print("📝 최종 결론")
print("=" * 60)
print(f"""
[분석 대상]
- 기간: 2022년 1분기 ~ 2025년 3분기 ({quarter_count}개 분기)
- 지역: 서울시 {dong_count}개 행정동
- 업종: 편의점

[데이터 규모]
- 총 데이터 수: {row_count:,}개
- 회귀분석 최소 기준(1,000개) 대비 {row_count/1000:.1f}배

[적합성 판단]
✅ 데이터 양: 적합 ({row_count:,}개 > 1,000개)
✅ 기간: 적합 ({quarter_count}개 분기 > 8개 분기)
✅ 결측치: 적합 (0%)

→ 회귀분석 진행에 충분한 데이터입니다! 🎉
""")